In [1]:
# Import the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
import seaborn as sns
from datetime import datetime
from datetime import date
import glob
import os
import matplotlib.pyplot as plt
import warnings
from pandas import read_csv

In [2]:
# Ignore warnings. This becomes important later in the code when all blank values have to be set to NAN
warnings.filterwarnings("ignore")

In [3]:
# Set the path to where the snotel data is stored
snotel_path = ('/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data')

In [4]:
# Search the path and extract the snotel files 
snotel_files = glob.glob(snotel_path +'/*.csv')
snotel_files

['/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data/Sacajawea_export.csv',
 '/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data/Brackett_export.csv']

In [5]:
# Create an empty list to hold all dataframes for the read in csv files
snotel_df_holder = []
# Create a for loop to cycle through all of the chena_files
for j in (snotel_files):
    # Create a new dataframe for each file
    df = pd.read_csv(j)
    # Change list of lists to a dataframe
    df = pd.DataFrame(df)
    station = os.path.basename(j)
    station = os.path.splitext(station)[0]
    station = station.replace('_export', '')
    # Create a new column in the dataframe to hold the year values
    df['station'] = station
    
    # Append the list of df_holder with all of the values for the individual dataframes
    snotel_df_holder.append(df)
    #snotel_df_holder = pd.DataFrame()
    snotel_df_holder
# Create a new dataframe that contains all of the dataframes created in the above loop
snotel_data_df = pd.concat(snotel_df_holder, axis = 0, ignore_index = False, join = 'outer')
# Create a new column titled 'Year' to hold the year derived from the datetime column
snotel_data_df['Year'] =  pd.DatetimeIndex(snotel_data_df['datetime']).year.astype(int)
# Do the same as above, but with Months
snotel_data_df['Month'] =  pd.DatetimeIndex(snotel_data_df['datetime']).month.astype(int)
# Change the format so that we only display year-month-day
snotel_data_df['datetime'] = pd.to_datetime(snotel_data_df['datetime']).dt.strftime('%Y-%m-%d')
# Check the dataframe to make sure the data has been read in correctly
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month
0,1999-09-30,NaN,NaN,37.76,43.34,32.00,NaN,Sacajawea,1999,9
1,1999-10-01,0.0,NaN,35.60,47.30,28.58,0.0,Sacajawea,1999,10
2,1999-10-02,0.0,NaN,28.40,31.64,24.62,0.0,Sacajawea,1999,10
3,1999-10-03,0.0,NaN,41.54,50.72,22.46,0.0,Sacajawea,1999,10
4,1999-10-04,0.0,NaN,50.54,61.16,36.86,0.0,Sacajawea,1999,10


In [6]:
# Make sure the datetime column is correctly identified as a pandas datetime with the format year-month-day
snotel_data_df['datetime'] = pd.to_datetime(snotel_data_df['datetime']).dt.strftime('%Y-%m-%d')
# Create a column to hold the 'Month' variable
snotel_data_df['Month'] =  pd.DatetimeIndex(snotel_data_df['datetime']).month.astype(int)
# Sort the dataframe by ascending date
snotel_data_df = snotel_data_df.sort_values(by = 'datetime', ascending = True)
# Check the dataframe and the datetime column to make sure this was done correctly
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month
0,1994-09-29,NaN,NaN,47.30,55.40,42.80,NaN,Brackett,1994,9
1,1994-09-30,NaN,NaN,46.04,53.06,40.82,NaN,Brackett,1994,9
2,1994-10-01,0.5,NaN,48.20,63.14,34.34,0.0,Brackett,1994,10
3,1994-10-02,0.0,NaN,27.86,36.14,19.22,0.0,Brackett,1994,10
4,1994-10-03,0.1,NaN,21.20,23.72,16.88,0.0,Brackett,1994,10


In [7]:
# View the average snow depth value
print('Average snow depth (in) ',snotel_data_df['Daily Snow Depth (in)'].mean())

Average snow depth (in)  19.00715153150722


In [8]:
# View the highest and lowest values of snow depth allowed based on 3 std away from the mean
print('Highest allowed',snotel_data_df['Daily Snow Depth (in)'].mean() + 3*snotel_data_df['Daily Snow Depth (in)'].std())
print('Lowest allowed',snotel_data_df['Daily Snow Depth (in)'].mean() - 3*snotel_data_df['Daily Snow Depth (in)'].std())

Highest allowed 85.22669811815294
Lowest allowed -47.2123950551385


In [9]:
# Filter the snow depth data by the highest and lowest allowed values
snotel_data_df['Daily Snow Depth (in)'] = np.where((0 < snotel_data_df['Daily Snow Depth (in)'])
                                                   & (snotel_data_df['Daily Snow Depth (in)'] <= 85.227), snotel_data_df['Daily Snow Depth (in)'], np.nan)
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month
0,1994-09-29,NaN,NaN,47.30,55.40,42.80,NaN,Brackett,1994,9
1,1994-09-30,NaN,NaN,46.04,53.06,40.82,NaN,Brackett,1994,9
2,1994-10-01,0.5,NaN,48.20,63.14,34.34,0.0,Brackett,1994,10
3,1994-10-02,0.0,NaN,27.86,36.14,19.22,0.0,Brackett,1994,10
4,1994-10-03,0.1,NaN,21.20,23.72,16.88,0.0,Brackett,1994,10


In [10]:
# Now, do the same for Daily Water Equivalent data
# View the highest and lowest values of snow water equivalent allowed based on 3 std away from the mean
print('Highest allowed',snotel_data_df['Daily Water Equivalent (in)'].mean() + 3*snotel_data_df['Daily Water Equivalent (in)'].std())
print('Lowest allowed',snotel_data_df['Daily Water Equivalent (in)'].mean() - 3*snotel_data_df['Daily Water Equivalent (in)'].std())

Highest allowed 29.739496501500387
Lowest allowed -17.198143118041738


In [11]:
# Filter the snow depth data by the highest and lowest allowed values
snotel_data_df['Daily Water Equivalent (in)'] = np.where((0 < snotel_data_df['Daily Water Equivalent (in)'])
                                                   & (snotel_data_df['Daily Water Equivalent (in)'] <= 29.739), snotel_data_df['Daily Water Equivalent (in)'], np.nan)
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month
0,1994-09-29,NaN,NaN,47.30,55.40,42.80,NaN,Brackett,1994,9
1,1994-09-30,NaN,NaN,46.04,53.06,40.82,NaN,Brackett,1994,9
2,1994-10-01,0.5,NaN,48.20,63.14,34.34,NaN,Brackett,1994,10
3,1994-10-02,0.0,NaN,27.86,36.14,19.22,NaN,Brackett,1994,10
4,1994-10-03,0.1,NaN,21.20,23.72,16.88,NaN,Brackett,1994,10


In [13]:
# Now, let's calculate snowpack bulk density
snotel_data_df['Density (%)'] = snotel_data_df['Daily Water Equivalent (in)']/ snotel_data_df['Daily Snow Depth (in)']*100
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month,Density (%)
0,1994-09-29,NaN,NaN,47.30,55.40,42.80,NaN,Brackett,1994,9,NaN
1,1994-09-30,NaN,NaN,46.04,53.06,40.82,NaN,Brackett,1994,9,NaN
2,1994-10-01,0.5,NaN,48.20,63.14,34.34,NaN,Brackett,1994,10,NaN
3,1994-10-02,0.0,NaN,27.86,36.14,19.22,NaN,Brackett,1994,10,NaN
4,1994-10-03,0.1,NaN,21.20,23.72,16.88,NaN,Brackett,1994,10,NaN


In [14]:
# SNOTEL data is not perfect, thus, we may get data records where the density is greater than 100%
# or less than 0. 
# This is erroneous data, thus must be set to NAN
snotel_data_df['Density (%)'] = np.where((snotel_data_df['Density (%)'] > 100) |(snotel_data_df['Density (%)'] < 0), 
                                               np.nan, snotel_data_df['Density (%)'])

In [15]:
# View the average density percentage
print('Average density %',snotel_data_df['Density (%)'].mean())

Average density % 30.174896171822937


In [16]:
# View the highest and lowest values allowed based on 3 std away from the mean
print('Highest allowed',snotel_data_df['Density (%)'].mean() + 3*snotel_data_df['Density (%)'].std())
print('Lowest allowed',snotel_data_df['Density (%)'].mean() - 3*snotel_data_df['Density (%)'].std())

Highest allowed 58.89417187266639
Lowest allowed 1.455620470979479


In [17]:
# Filter the snotel Density data to exclude the low and high limits of density
snotel_data_df['Density (%)'] = np.where((1.456 <= snotel_data_df['Density (%)']) & (snotel_data_df['Density (%)'] <= 58.894), snotel_data_df['Density (%)'], np.nan)

In [18]:
# It can be beneficial to view the density data in terms of kg/m^3 since this is the unit density is reported in 
# with snow science
snotel_data_df['Density (kg/m3)'] = snotel_data_df['Density (%)'] * 10

In [19]:
# Create new columns to hold data converted to metric
snotel_data_df['Daily Precipitation (cm)'] = snotel_data_df['Daily Precipitation (in)']*2.54
snotel_data_df['Daily Snow Depth (cm)'] = snotel_data_df['Daily Snow Depth (in)']*2.54
snotel_data_df['Daily Water Equivalent (cm)'] = snotel_data_df['Daily Water Equivalent (in)']*2.54
snotel_data_df['Daily Average Temp (C)'] = (snotel_data_df['Daily Average Temperature (F)'] - 32) *5/9
# View the data to make sure the new columns have updated
snotel_data_df.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month,Density (%),Density (kg/m3),Daily Precipitation (cm),Daily Snow Depth (cm),Daily Water Equivalent (cm),Daily Average Temp (C)
0,1994-09-29,NaN,NaN,47.30,55.40,42.80,NaN,Brackett,1994,9,NaN,NaN,NaN,NaN,NaN,8.5
1,1994-09-30,NaN,NaN,46.04,53.06,40.82,NaN,Brackett,1994,9,NaN,NaN,NaN,NaN,NaN,7.8
2,1994-10-01,0.5,NaN,48.20,63.14,34.34,NaN,Brackett,1994,10,NaN,NaN,1.270,NaN,NaN,9.0
3,1994-10-02,0.0,NaN,27.86,36.14,19.22,NaN,Brackett,1994,10,NaN,NaN,0.000,NaN,NaN,-2.3
4,1994-10-03,0.1,NaN,21.20,23.72,16.88,NaN,Brackett,1994,10,NaN,NaN,0.254,NaN,NaN,-6.0


In [20]:
# Create a new dataframe the contains the metric data
snotel_metric = snotel_data_df[['datetime','station','Year','Month','Daily Precipitation (cm)','Daily Snow Depth (cm)','Daily Water Equivalent (cm)',
                                'Daily Average Temp (C)','Density (%)','Density (kg/m3)']]
# View the new dataframe
snotel_metric.head()

,datetime,station,Year,Month,Daily Precipitation (cm),Daily Snow Depth (cm),Daily Water Equivalent (cm),Daily Average Temp (C),Density (%),Density (kg/m3)
0,1994-09-29,Brackett,1994,9,NaN,NaN,NaN,8.5,NaN,NaN
1,1994-09-30,Brackett,1994,9,NaN,NaN,NaN,7.8,NaN,NaN
2,1994-10-01,Brackett,1994,10,1.270,NaN,NaN,9.0,NaN,NaN
3,1994-10-02,Brackett,1994,10,0.000,NaN,NaN,-2.3,NaN,NaN
4,1994-10-03,Brackett,1994,10,0.254,NaN,NaN,-6.0,NaN,NaN


In [21]:
# Reset the index to make sure the new dataframe does not have erroneous data
snotel_metric = snotel_metric.reset_index(drop = True)

In [22]:
# Export the metric dataframe to our snotel data > site data folder
snotel_metric.to_csv('/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data/Cleaned_Data/snotel_metric_df.csv')

In [23]:
# Export the imperial dataframe that is cleaned
snotel_data_df.to_csv('/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data/Cleaned_Data/snotel_cleaned_df.csv')

In [32]:
# Filter the data by the 2021-22 field season
field_season = snotel_data_df.loc[(snotel_data_df['datetime'] > '2021-10-01') & (snotel_data_df['datetime'] < '2022-07-01')]
field_season.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month,Density (%),Density (kg/m3),Daily Precipitation (cm),Daily Snow Depth (cm),Daily Water Equivalent (cm),Daily Average Temp (C)
9865,2021-10-02,0.0,NaN,51.08,60.98,40.10,NaN,Brackett,2021,10,NaN,NaN,0.0,NaN,NaN,10.6
8038,2021-10-02,0.0,NaN,49.46,62.42,35.78,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,9.7
8039,2021-10-03,0.0,NaN,55.76,66.20,44.06,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,13.2
9866,2021-10-03,0.0,NaN,55.58,64.40,47.66,NaN,Brackett,2021,10,NaN,NaN,0.0,NaN,NaN,13.1
8040,2021-10-04,0.0,NaN,56.66,71.60,41.36,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,13.7


In [35]:
field_season = field_season.reset_index(drop = True)
# field_season.index = pd.to_datetime(field_season.index)
field_season.head()

,datetime,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in),station,Year,Month,Density (%),Density (kg/m3),Daily Precipitation (cm),Daily Snow Depth (cm),Daily Water Equivalent (cm),Daily Average Temp (C)
0,2021-10-02,0.0,NaN,51.08,60.98,40.10,NaN,Brackett,2021,10,NaN,NaN,0.0,NaN,NaN,10.6
1,2021-10-02,0.0,NaN,49.46,62.42,35.78,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,9.7
2,2021-10-03,0.0,NaN,55.76,66.20,44.06,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,13.2
3,2021-10-03,0.0,NaN,55.58,64.40,47.66,NaN,Brackett,2021,10,NaN,NaN,0.0,NaN,NaN,13.1
4,2021-10-04,0.0,NaN,56.66,71.60,41.36,NaN,Sacajawea,2021,10,NaN,NaN,0.0,NaN,NaN,13.7


In [36]:
# Export the field season dataframe
field_season.to_csv('/Users/f67f911/Desktop/Bridger_snotels/Data/Site_Data/Cleaned_Data/field_season_df.csv')